# PLAID to I.DOT notebook

In [369]:
# load modules and select input PLAID file
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import glob
import os
import math
import re
import datetime
import string
from ast import literal_eval

from ipynb.fs.full.my_functions import *

import warnings

# select input files

In [370]:
PLAID_layout  = "input/bf-moa-live-60-PLAID.csv"
compound_info = "input/compound_info.csv"

In [371]:
# merge multiple files
#...
#

In [372]:
# add DMSO to all empty wells
# ....


In [373]:
user_name             = 'Jonne' #@param ["Jonne","Amelie","Christa","Malin","Martin","Polina","Axel","David","Ovidiu"]
protocol_name         = "bf-moa-live" #@param {type:"string"} # Name of the protocol

# more details: https://dispendix.com/idot-dispensing-plates/
sourceplate_type      = "S.100 Plate" #@param ["S.100 Plate","S.60 Plate","S.200 Plate"] 
target_plate_type     = 'MWP 384' #@param ["MWP 96","MWP 384"]

working_volume_ul     = 40 #@param {type:"number"}

x = datetime.datetime.now()

software      = "1.7.2021.1019" # I-DOT Assay Studio software version
user_name     = user_name # User name/ID
date          = (x.strftime("%x")) 
time          = (x.strftime("%X")) 


max_volume       = 8.0E-5 # Source plate max volume (80000nL = 80uL= 8.0E-5 L)
waste            = "Waste Tube" # Position of the waste well on the target carrier

dispense_to_waste          = True # Enable/disable priming before dispensing (=True/False) 
dispense_to_waste_cycles   = 2    # Number of priming cycles for each source well (=1/2/3)
dispense_to_waste_volume   = 5e-8 # Dispensing volume for each priming cycle (=5e-8/.../1e-6) 
use_deionisation           = True
optimization_level         = "ReorderAndParallel" # Used protocol optimization process to reduce total dispensing time. Possible values are NoOptimization / Reorder / ReorderAndParalell
waste_error_handling_level = "Ask" # Checkpoint for the dispensing run if no droplets are detected during priming. Possible values are Ask / Abort / Continue 
save_liquids               = "Ask" # Checkpoint for Liquid Library handling. Possible options are Ask / Never


# show compounds, doses and replicates

In [374]:
df = pd.read_csv(PLAID_layout, dtype={'cmpdname': object} )

replicates = df.groupby(["cmpdname","CONCuM"]).size().to_frame('counts')
replicates.head(2)

,,counts
cmpdname,CONCuM,
DMSO,0,396
c1,10,12


# Optional preprocessing steps

### Change dose names in PLAID? (optional)

In [375]:
change_concnames_PLAID         = False #@param {type:"boolean"}

if change_concnames_PLAID == True:
  #newconc = {"a": 0.001, "b": 0.01, "c": 0.1, "d": 1, "e": 10} # here you assign the new names of choice! 
  newconc = {"a": "[0.01,1]", "b":"[0.1,1]", "c":"[1,1]", "d":"[10,1]"}
  df = df.replace({"CONCuM": newconc})

### Change compound names in PLAID? (optional)

In [376]:
change_cmpdnames_PLAID         = False #@param {type:"boolean"}

if change_cmpdnames_PLAID == True:
      print("yes")
      newcmpd = {"1":"['Nir','Car']"}
      df = df.replace({"cmpdname": newcmpd})
        
replicates = df.groupby(["plateID","well"])["cmpdname"].apply(lambda x: ','.join(x)).to_frame("cmpdname") # account for combinations!
replicates = replicates.reset_index()
replicates = replicates.groupby(["cmpdname"]).sum().reset_index()                  
replicates = replicates[["cmpdname"]].value_counts().to_frame('counts')
#replicates.head(5)

### Dissolve combinations (optional)

In [377]:
reformat_combinations         = False #@param {type:"boolean"}

if reformat_combinations == True:

  df_combinations = df.copy()
  df_combinations = df[df.cmpdname.str.match(r'[\[*,*\]]')] # subset only rows with combinations
  df_singles      = df[~df.cmpdname.str.match(r'[\[*,*\]]')] # subset rows without combinations
  print(len(df_combinations), "combinations")
  print(len(df_singles), "single compounds")

  #print(df_combinations["cmpdname"].apply(type)) # check datatype
  df_combinations["cmpdname"]  = df_combinations["cmpdname"].apply(literal_eval) #convert to list type
  df_combinations["CONCuM"]   = df_combinations["CONCuM"].apply(literal_eval)    #convert to list type

  df_combinations = df_combinations.explode(["cmpdname", "CONCuM"])

  df = pd.concat([df_combinations, df_singles], ignore_index=True, sort=False)

  #df

### Change units (optional)

In [378]:
# default is uM - here we will recalculate if you have put in another format

unit                  = "uM" #@param ["pM","nM","uM","mM","dilution"]

# 1 Micromolar [µM] = 0.001 Millimolar [mM]
# 1 Micromolar [µM] = 1000 Nanomolar [nM]
# 1 Micromolar [µM] = 1.0×106 Picomolar [pM] (1000000)


df['CONCuM'] = pd.to_numeric(df['CONCuM'], errors='coerce')

if unit == "uM":
  None
elif unit == "pM":
  df['CONCuM'] =  df["CONCuM"]/ 1000000
elif unit == "nM":
  df['CONCuM'] = df["CONCuM"]/ 1000
elif unit == "mM":
  df['CONCuM'] = df["CONCuM"] * 1000
elif unit == "dilution":
  df['CONCuM'] = df["CONCuM"] * 1000


# assign compound details

In [379]:
compound = pd.read_csv(compound_info, dtype={'cmpdname': object}, usecols=["cmpdname","solvent","source_plate","source_well","source_plate","highest_stock_mM"])
compound.head(2)

,highest_stock_mM,solvent,source_plate,source_well,cmpdname
0,10.0,DMSO,source10mM,A02,c1
1,10.0,DMSO,source10mM,A04,c2


In [380]:
merged_df = pd.merge(df, compound, on='cmpdname', how='outer', indicator=True)
    
# Checking for unmatched rows
unmatched_rows = merged_df[merged_df['_merge'] != 'both']

if not unmatched_rows.empty:
    print('Warning: there are unmatched rows in the merged DataFrame')
else:
    print("all mcompounds are matched")

all mcompounds are matched


In [381]:
df = merged_df.copy()
df.head(2)

,plateID,well,r,c,cmpdname,CONCuM,cmpdnum,highest_stock_mM,solvent,source_plate,source_well,_merge
0,L1,B02,B,2,c1,10,c1_10,10.0,DMSO,source10mM,A02,both
1,L1,I18,I,18,c1,10,c1_10,10.0,DMSO,source10mM,A02,both


# Assign treatment groups

In [382]:
# add treatment group based on controls, treatments, blanks and DMSO wells

cmpdname                   = np.unique(df[['cmpdname']].values).tolist()
catdict                    = treatmentdict(cmpdname)
df["treatment_type"]       = df["cmpdname"].map(catdict)
df["treatment_type"].value_counts().to_frame('number of wells')

,number of wells
trt,648
DMSO,396


In [383]:
# mask compiund that do not have water
df = df[df["solvent"] != "water"]
mask_compounds      = np.unique(df["cmpdname"])
mask_compounds_list = mask_compounds.tolist()

# Assign highest stock concentration

In [384]:
# advanced settings: retrieve stock concentrations from compound list
manual_list_stocks   = True #@param {type:"boolean"}

In [385]:
if manual_list_stocks == False:
    
    ctrl_highest_stock_mM = 10
    trt_highest_stock_mM  = 10
    
    df["highest_stock_mM"] = df["treatment_type"].apply(lambda x: ctrl_highest_stock_mM if x == 'ctrl' else trt_highest_stock_mM if x == 'trt' else None)

In [386]:
if manual_list_stocks == True: 

    higheststock = pd.read_csv(compound_info, dtype={'cmpdname': object}, usecols=["cmpdname","highest_stock_mM"])
    manualstocksdict = dict(zip(higheststock.cmpdname, higheststock.highest_stock_mM))
    df["highest_stock_mM"]       = df["cmpdname"].map(manualstocksdict)

# check for NaN values 
if df['highest_stock_mM'].isna().any():
    warnings.warn("Warning: NaN values found in 'highest_stock_mM'!")

# plot count for each stock concentration
df[["highest_stock_mM"]].value_counts().to_frame('counts')

,counts
highest_stock_mM,
10.0,600
0.0,396


# Assign suitable stock concentration to each well

In [387]:
max_dmso                   = 1 #@param {type:"number"}

df["stock_conc_mM"]        = df.apply(lambda x: stockfinder(x['CONCuM'],
                                                            x["highest_stock_mM"],
                                                            V2_ul= working_volume_ul,
                                                            sourceplate_type=sourceplate_type,
                                                            dmso_percmax=max_dmso),
                                                            axis=1)


In [388]:
# list of concentrations by user

# plot count for each stock concentration
df[["stock_conc_mM"]].value_counts().to_frame('counts')

,counts
stock_conc_mM,
10,600
0,396


# Calculate volumes for spotting

In [389]:
def uLfromstock(concUM, stock_conc_mM, working_volume_ul):
    concUM =  (concUM * working_volume_ul) / stock_conc_mM if stock_conc_mM != 0 else 0
    return concUM / 1000

df["Volume [uL]"]          = df.apply(lambda x: uLfromstock(x["CONCuM"], x["stock_conc_mM"], working_volume_ul), axis=1)


# warn if volume is higher than plate max
highestvolume = df["Volume [uL]"].max()
print("highest volume in plate:",highestvolume, "ul")

wellcapacity = int(max_volume *1e06) # wellcapacity based on idot plate
if highestvolume > wellcapacity:
  raise Exception("The volume needed for some wells (highestvolume) exceeds the I.DOT well capacity (80ul), revise your setup !!") 

# we also reformat some labels to work with the idot 
  
df["Target Well"]          = df["well"].apply(removeleadingzero)
df["Liquid Name"]          = df['cmpdname'] + "[" +  df['stock_conc_mM'].astype(str) + "]"
df.rename(columns          = {'plateID':'Target Plate'}, inplace = True)
df                         = df[["Target Plate","cmpdname","highest_stock_mM","stock_conc_mM","treatment_type","Target Well","Liquid Name","Volume [uL]"]]


highest volume in plate: 0.04 ul


# Normalize plates 

In [390]:
solvent              = "DMSO"
DMSOstrat            = "To fixed percentage" #@param ["None", "To highest in plates", "To fixed percentage"]
dmso_fixed_percmax   = 0.2 #@param {type:"number"}

In [391]:
mask                   = df['cmpdname'].isin(mask_compounds_list)
df_to_normalize        = df[mask] # rows that should not be normalized

In [392]:
grouped_df = df_to_normalize.groupby(["Target Plate","Target Well"]).sum().reset_index() # account for combinations!

if DMSOstrat == "None":
  maxDMSO = 0
    
elif DMSOstrat == "To highest in plates":
  maxDMSO            = grouped_df.loc[grouped_df["Volume [uL]"].idxmax()]
  maxDMSO            = maxDMSO["Volume [uL]"]
    
elif DMSOstrat == "To fixed percentage":
  maxDMSOfrac    = dmso_fixed_percmax / 100
  maxDMSO        = maxDMSOfrac * working_volume_ul 

In [393]:
grouped_df

,Target Plate,Target Well,highest_stock_mM,stock_conc_mM,Volume [uL]
0,L1,A1,0.0,0,0.0
1,L1,A10,0.0,0,0.0
2,L1,A11,0.0,0,0.0
3,L1,A12,0.0,0,0.0
4,L1,A13,0.0,0,0.0
...,...,...,...,...,...
991,L3,P5,0.0,0,0.0
992,L3,P6,0.0,0,0.0
993,L3,P7,0.0,0,0.0
994,L3,P8,0.0,0,0.0


In [394]:
dfDMSO = normalizeDMSO(grouped_df, maxDMSO= dmso_fixed_percmax)

print("DMSO in each well will be filled up to:", maxDMSO ,"ul DMSO, corresponding to: ", maxDMSO/working_volume_ul*100, "% DMSO")
print("A total of",len(dfDMSO), "wells are normalized")

DMSO in each well will be filled up to: 0.08 ul DMSO, corresponding to:  0.2 % DMSO
A total of 996 wells are normalized


In [395]:
# Merge target plates

frames      = [df] 
target      = pd.concat(frames)
target      = target[target["Volume [uL]"] != 0]

In [396]:
target.head(3)

,Target Plate,cmpdname,highest_stock_mM,stock_conc_mM,treatment_type,Target Well,Liquid Name,Volume [uL]
0,L1,c1,10.0,10,trt,B2,c1[10],0.04
1,L1,c1,10.0,10,trt,I18,c1[10],0.04
2,L1,c1,10.0,10,trt,K14,c1[10],0.04


# -- -- -- -- -- -- -- -- -- -- -- -- -- --
# Now we will generate source plates
# -- -- -- -- -- -- -- -- -- -- -- -- -- --

In [397]:
sourceplate_strat            = 'manual_list' #@param ["automation", "simple", "manual_list"]
seperate_batches_8           = False #@param {type:"boolean"}

In [398]:
# Assign DMSO source well and generate DMSO source plate

dfvolumes             = dfDMSO["Volume [uL]"].to_list()
dfDMSO["Source Well"] = assign_DMSOsource(max_volume, dfvolumes)
dfDMSO[['Liquid Name','Source Plate']] = 'DMSO', 'DMSOsource'
dmsoSOURCE            = dfDMSO[["Liquid Name","Source Plate","Source Well"]].drop_duplicates()

### DMSO source

In [399]:
#@title Visualize and save DMSO Source plates

if len(dmsoSOURCE.columns) == 0:
  print('no controls source plate was created')
else:
  DMSOlist = dmsoSOURCE['Source Plate'].unique().tolist()
  for plate in DMSOlist:
      df = dmsoSOURCE[dmsoSOURCE['Source Plate'] == plate ] 
      df['Metadata_Column'] =    df['Source Well'].astype(str).str[1:3]
      df['Metadata_Row']    =    df['Source Well'].astype(str).str[0]
      DMSOpivot = df.pivot(columns="Metadata_Column", index="Metadata_Row", values="Liquid Name") 
      print("controls source plate:") 
      DMSOpivot 

      DMSOpivot.to_csv("{}_{}.csv".format(protocol_name,plate), encoding = 'utf-8-sig') 

DMSOpivot

controls source plate:


Metadata_Column,01
Metadata_Row,
A,DMSO
B,DMSO
C,DMSO


### treatment source place

In [400]:
# Option 1: simple (Generate source plates for control and treatment compounds)

# simple placement of compounds from start to end

if sourceplate_strat=="simple":
  trtSOURCE = assignsourcesimple([target])

In [401]:
# Option 2: Automation friendly source compound allocations. (generate source plates for treatment and controls)

if sourceplate_strat=="automation":
    trtSOURCE  = assignsource_automated(target,"trt")
    ctrlSOURCE  = assignsource_automated(target,"ctrl")

In [402]:
# Option 3: Manual source compound allocation (step 1: download compound list)

if sourceplate_strat == 'manual_list':
    
    compounds_df          = target[["Liquid Name", "cmpdname","stock_conc_mM"]].drop_duplicates()
    compounds_df[['Source Well','Source Plate']] = ''    
    compounds_df          = compounds_df[~compounds_df['Liquid Name'].str.contains('DMSO')]
    
    # append the actual DMSO 
    dmsoSOURCE            = dmsoSOURCE[["Liquid Name"]]
    dmsoSOURCE[["cmpdname","stock_conc_mM",'Source Well','Source Plate']] = ''  
    compounds_df          = pd.concat([compounds_df, dmsoSOURCE], ignore_index=True)
    
    compounds_df.to_csv("user_input/manual_source_list.csv", index=False, encoding = 'utf-8-sig') 
    #files.download("manual_source_list.csv")

In [404]:
#@title Option 3: Manual source compound allocation (step 3: read manual compound list)
if sourceplate_strat == 'manual_list':
    
    manual_source = pd.read_csv("user_input/manual_source_list_completed.csv")
    
    # check input
    if manual_source.duplicated(subset=["Source Plate","Source Well"]).any():
        raise Exception('Duplicate wells found in the dataframe.')

    elif not all(re.match(r'^[A-H]\d{2}$', value) for value in manual_source['Source Well']):
        raise Exception('Source well does not match the expected format.')
        
    else:
        trtSOURCE     = manual_source[["Liquid Name","Source Well","Source Plate"]]

trtlist = trtSOURCE['Source Plate'].unique().tolist()
print("the treatment source plates included:", trtlist)

the treatment source plates included: [1]


In [405]:
#@title Visualize and save treatment Source plates

if len(trtSOURCE.columns) == 0:
  print('no controls source plate was created')
else:
  trtlist = trtSOURCE['Source Plate'].unique().tolist()
  for plate in trtlist:
        df = trtSOURCE[trtSOURCE['Source Plate'] == plate ] 
        df['Metadata_Column'] =    df['Source Well'].astype(str).str[1:3]
        df['Metadata_Row']    =    df['Source Well'].astype(str).str[0]

        # save alist with source wells
        df.to_csv("{}_{}.csv".format("list_source",plate), encoding = 'utf-8-sig', index=False) 
        #files.download("{}_{}.csv".format("list_source",plate))

        # pivot table for easier visualization
        trtpivot = df.pivot(columns="Metadata_Column", index="Metadata_Row", values="Liquid Name") 
        print("treatment source plate:", plate) 
        trtpivot.to_csv("output/{}_{}.csv".format(protocol_name,plate), encoding = 'utf-8-sig', index=False) 
        #files.download("{}_{}.csv".format(protocol_name,plate))

print("here a preview of the last source plate:")
trtpivot

treatment source plate: 1
here a preview of the last source plate:


Metadata_Column,01,02,03,04,05,06,07,08,09,10,11,12
Metadata_Row,,,,,,,,,,,,
A,c1[10],c27[10],c4[10],c33[10],c39[10],c7[10],c47[10],c30[10],c5[10],c48[10],c10[10],c13[10]
B,c45[10],c42[10],c25[10],c22[10],c40[10],c9[10],c14[10],c36[10],c43[10],c20[10],c37[10],c21[10]
C,c38[10],c18[10],c17[10],c44[10],c35[10],c19[10],c34[10],c31[10],c11[10],c32[10],c12[10],c3[10]
D,c16[10],c54[10],c41[10],c2[10],c46[10],c26[10],c53[10],c8[10],c23[10],c29[10],c24[10],c15[10]
E,c6[10],c28[10],DMSO,DMSO,DMSO,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [406]:
#@title Visualize and save controls Source plates

#if len(ctrlSOURCE.columns) == 0:
#  print('no controls source plate was created, if this is correct just continue')

if 'ctrlSOURCE' not in locals() or 'ctrlSOURCE' not in globals():
    print('no controls source plate was created, if this is correct just continue')

else:
  ctrllist = ctrlSOURCE['Source Plate'].unique().tolist()
  for plate in ctrllist:
      df = ctrlSOURCE[ctrlSOURCE['Source Plate'] == plate ] 
      df['Metadata_Column'] =    df['Source Well'].astype(str).str[1:3]
      df['Metadata_Row']    =    df['Source Well'].astype(str).str[0]
      ctrlpivot = df.pivot(columns="Metadata_Column", index="Metadata_Row", values="Liquid Name") 
      print("controls source plate were created:", plate) 
      ctrlpivot 

      ctrlpivot.to_csv("output/{}_{}.csv".format(protocol_name,plate), encoding = 'utf-8-sig') 
      #files.download("{}_{}.csv".format(protocol_name,plate))

no controls source plate was created, if this is correct just continue


In [407]:
# merge all source plates

if sourceplate_strat == 'manual_list':
  frames = [trtSOURCE]
else:
  frames = [trtSOURCE, ctrlSOURCE, dmsoSOURCE]
  
source = pd.concat(frames)

#source

In [408]:
# change leading zeros
if source["Source Well"].str.len().min() > 2:
    source['Source Well']= source['Source Well'].apply(removeleadingzero)

# format protocols

In [409]:
combine_to_one_protocol  = True # Enable/disable priming before dispensing (=True/False) 

In [410]:
# Next step is to create and format the idot protocol

# assign source well for targets
sourcedictwell  = dict(source[["Liquid Name","Source Well"]].values)
target["Source Well"]  = target["Liquid Name"].map(sourcedictwell)

# assign source plate for targets
sourceplatedict = dict(source[["Liquid Name","Source Plate"]].values)
target["Source Plate"] = target["Liquid Name"].map(sourceplatedict)

sourceplates = source['Source Plate'].unique().tolist()
targetplates = target['Target Plate'].unique().tolist()

#sourceplates = sourceplates[::-1] # ?

targetformat = target[["Target Plate","Target Well","Volume [uL]","Liquid Name"]]

print(target["Liquid Name"].dtypes)

print(sourceplates)
print(targetplates)

object
[1]
['L1', 'L2', 'L3']


In [411]:
# code for 1 protocol per source plate

if combine_to_one_protocol == False:
    
    for splate in sourceplates:
        collected_df = []
        i = 1

        for tplate in targetplates:
            df = target.loc[((target["Source Plate"] == splate) & (target["Target Plate"] == tplate ) )]

            df = df[["Source Well","Target Well","Volume [uL]","Liquid Name"]]
            df = df.reindex(columns=[*df.columns.tolist(), "", "","",""], fill_value="")

            df = pd.concat([df.columns.to_frame().T, df], ignore_index=True)
            df.columns = range(len(df.columns)) 

            subheader = [[sourceplate_type, splate, "",max_volume, target_plate_type, tplate, "",waste],
            ["DispenseToWaste="+str(dispense_to_waste),"DispenseToWasteCycles="+str(dispense_to_waste_cycles),"DispenseToWasteVolume="+str(dispense_to_waste_volume),"UseDeionisation="+str(use_deionisation),"OptimizationLevel="+str(optimization_level),"WasteErrorHandlingLevel="+str(waste_error_handling_level),"SaveLiquids="+str(save_liquids),""]]
            subheader = pd.DataFrame(subheader)

            protocol = pd.concat([subheader, df],ignore_index=True)
            collected_df.append(protocol)

        dfs = pd.concat(collected_df)

        header = [[protocol_name, software, user_name, date, time,"","",""]]
        header = pd.DataFrame(header)

        # header and pipetting steps:
        fullprotocol = pd.concat([header, dfs],ignore_index=True)

        fullprotocol.to_csv("IDOT_{}.csv".format(splate), header=False, index=False,encoding = 'utf-8-sig') 
        #files.download("IDOT_{}.csv".format(splate))
    

In [412]:
#@title Next step is to create and format the idot protocol (all in 1 protocol)

if combine_to_one_protocol == True:
    
    collected_df = []
    i = 1

    for splate in sourceplates:
        for tplate in targetplates:
            df = target.loc[((target["Source Plate"] == splate) & (target["Target Plate"] == tplate ) )]

            df = df[["Source Well","Target Well","Volume [uL]","Liquid Name"]]
            df = df.reindex(columns=[*df.columns.tolist(), "", "","",""], fill_value="")

            df = pd.concat([df.columns.to_frame().T, df], ignore_index=True)
            df.columns = range(len(df.columns)) 

            subheader = [[sourceplate_type, splate, "",max_volume, target_plate_type, tplate, "",waste],
            ["DispenseToWaste="+str(dispense_to_waste),"DispenseToWasteCycles="+str(dispense_to_waste_cycles),"DispenseToWasteVolume="+str(dispense_to_waste_volume),"UseDeionisation="+str(use_deionisation),"OptimizationLevel="+str(optimization_level),"WasteErrorHandlingLevel="+str(waste_error_handling_level),"SaveLiquids="+str(save_liquids),""]]
            subheader = pd.DataFrame(subheader)

            protocol = pd.concat([subheader, df],ignore_index=True)
            collected_df.append(protocol)

    header = [[protocol_name, software, user_name, date, time,"","",""]]
    header = pd.DataFrame(header)

    dfs = pd.concat(collected_df)
    fullprotocol = pd.concat([header, dfs],ignore_index=True)

    fullprotocol.to_csv("IDOT_{}.csv".format(protocol_name), header=False, index=False,encoding = 'utf-8-sig') 
    #files.download("IDOT_{}.csv".format(protocol_name))


In [413]:
# see the protocol
fullprotocol

,0,1,2,3,4,5,6,7
0,bf-moa-live,1.7.2021.1019,Jonne,03/25/23,22:18:50,,,
1,S.100 Plate,1,,0.00008,MWP 384,L1,,Waste Tube
2,DispenseToWaste=True,DispenseToWasteCycles=2,DispenseToWasteVolume=5e-08,UseDeionisation=True,OptimizationLevel=ReorderAndParallel,WasteErrorHandlingLevel=Ask,SaveLiquids=Ask,
3,Source Well,Target Well,Volume [uL],Liquid Name,,,,
4,A1,B2,0.04,c1[10],,,,
...,...,...,...,...,...,...,...,...
605,E1,J21,0.04,c6[10],,,,
606,E2,D18,0.04,c28[10],,,,
607,E2,I6,0.04,c28[10],,,,
608,E2,K23,0.04,c28[10],,,,


# Cross-check all conditions from protocol

In [414]:
sourcewell = source[["Liquid Name","Source Well"]]
sourcedictwell = dict(sourcewell.values)

sourceplate = source[["Liquid Name","Source Plate"]]
sourceplatedict = dict(sourceplate.values)

target["Source Well"]  = target["Liquid Name"].map(sourcedictwell)
target["Source Plate"] = target["Liquid Name"].map(sourceplatedict)

sourceplates = source['Source Plate'].unique().tolist()
targetplates = target['Target Plate'].unique().tolist()

sourceplates = sourceplates[::-1]

targetformat = target[["Target Plate","Target Well","Volume [uL]","Liquid Name"]]

collected_df = []
i = 1

In [415]:
#@title Cross-check all liquids to check

def split_column(text):
    if '[' in text:
        text_split = text.split('[')
        return pd.Series([text_split[0], text_split[1].rstrip(']')])
    else:
        return pd.Series([text, ''])

for splate in sourceplates:
    for tplate in targetplates:
        df = target.loc[((target["Source Plate"] == splate) & (target["Target Plate"] == tplate ) )]                
        df = df[["Source Well","Target Well","Volume [uL]","Liquid Name"]]
        df[['Compound_Name', 'Compound_Dose']] = df['Liquid Name'].apply(split_column)
        collected_df.append(df)

df2 = pd.concat(collected_df)

df2

,Source Well,Target Well,Volume [uL],Liquid Name,Compound_Name,Compound_Dose
0,A1,B2,0.04,c1[10],c1,10
1,A1,I18,0.04,c1[10],c1,10
2,A1,K14,0.04,c1[10],c1,10
3,A1,N11,0.04,c1[10],c1,10
12,A2,B3,0.04,c27[10],c27,10
...,...,...,...,...,...,...
635,E1,J21,0.04,c6[10],c6,10
644,E2,D18,0.04,c28[10],c28,10
645,E2,I6,0.04,c28[10],c28,10
646,E2,K23,0.04,c28[10],c28,10


In [416]:
replicates = df2.groupby(["Liquid Name","Volume [uL]"]).size().to_frame('counts')
replicates

,,counts
Liquid Name,Volume [uL],
c10[10],0.04,12
c11[10],0.04,12
c12[10],0.04,12
c13[10],0.04,12
c14[10],0.04,12
c15[10],0.04,12
c16[10],0.04,12
c17[10],0.04,12
c18[10],0.04,12


In [417]:
#@title Save a sumary of all pipetting steps

df2.to_csv("output/IDOT_summary{}.csv".format(protocol_name), index=False,encoding = 'utf-8-sig') 
#files.download("final_steps{}.csv".format(protocol_name))